### Networkx To Age

In [1]:
import psycopg2
from age.networkx import *

conn = psycopg2.connect(
    host="localhost",
    port="5430",
    dbname="postgresDB",
    user="postgresUser",
    password="postgresPW")
graphName = 'bitnine_global_inic'

age.setUpAge(conn, graphName)
age.deleteGraph(conn,graphName)
age.setUpAge(conn, graphName)

In [2]:
import random

num_nodes = 10000
num_edges = 50000
G = nx.DiGraph()
try:
    for i in range(num_nodes):
        G.add_node(i, label='Number')    
    for i in range(num_edges):
        source = random.randint(0, num_nodes-1)
        target = random.randint(0, num_nodes-1)
        G.add_edge(source, target, label='Connection')
except Exception as e:
    raise e

In [3]:
networkx_to_age(conn, G, graphName)

### AGE to Networkx

In [4]:
age.setUpAge(conn, graphName)
age.deleteGraph(conn,graphName)
age.setUpAge(conn, graphName)

with conn.cursor() as cursor:
    cursor.execute("""
    SELECT * from cypher('%s', $$
    CREATE 
    (:User {id: 'user1'}),
    (:User {id: 'user2'}),
    (:Movie {id: 'movie1', name: 'The Shawshank Redemption'}),
    (:Movie {id: 'movie2', name: 'The Godfather'}),
    (:Movie {id: 'movie3', name: 'The Dark Knight'}),
    (:Movie {id: 'movie4', name: 'Army of Thieves'}),
    (:Movie {id: 'movie5', name: 'Oceans Thirteen'}),
    (:Movie {id: 'movie6', name: 'Heat'}),
    (:Genre {name: 'Drama'}),
    (:Genre {name: 'Crime'}),
    (:Genre {name: 'Action'})
    $$) as (V agtype);
    """ % (graphName))
    conn.commit()

with conn.cursor() as cursor:
    cursor.execute("""
    SELECT * from cypher('%s', $$
    MATCH 
    (u1:User {id: 'user1'}), 
    (u2:User {id: 'user2'}), 
    (m1:Movie {id: 'movie1'}),
    (m2:Movie {id: 'movie2'}),
    (m3:Movie {id: 'movie3'}),
    (m4:Movie {id: 'movie4'}),
    (m5:Movie {id: 'movie5'}),
    (m6:Movie {id: 'movie6'}),
    (gD:Genre {name: 'Drama'}),
    (gC:Genre {name: 'Crime'}),
    (gA:Genre {name: 'Action'})
    CREATE 
    (u1)-[:WATCHED]->(m1),
    (u1)-[:WATCHED]->(m2),
    (u2)-[:WATCHED]->(m1),
    (m1)-[:BELONGS_TO]->(gD),
    (m2)-[:BELONGS_TO]->(gC),
    (m3)-[:BELONGS_TO]->(gA),
    (m4)-[:BELONGS_TO]->(gD),
    (m5)-[:BELONGS_TO]->(gC),
    (m6)-[:BELONGS_TO]->(gA)
    $$) as (V agtype);
    """ % (graphName))
    conn.commit()


In [5]:
G = age_to_networkx(conn, graphName)

Found 11 nodes and 9 edges
